<a href="https://colab.research.google.com/github/mrklees/learning-agents/blob/master/2_The_2_armed_Bandit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducing the Problem

In this notebook we [follow along with Arthur](https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-1-fd544fab149) on the famous [multi-armed bandit problem.](https://en.wikipedia.org/wiki/Multi-armed_bandit)  While I frequently dislike working on contrived problems, the multi-armed bandit is a classical problem for the reason that it's very analgous to many decisions we have to make.  The multi-armed bandit problem demonstrates the exploration vs exploitation trade off, and helps us to think about how our agents might tackle such a problem.

Something of note.  The basest explanation of our agent's objective is to find the set of actions which maximizes some reward function.  The challenge is that rewards can vary by multiple conditions including:
* Rewards can vary by the action you take.
* Rewards can be delayed, meaning you might not know immediately if you made a good or bad decision.
* Rewards can be conditional on the state of the environment. 

Multi-armed bandit only suffers from the first problem, as each arm is defined as having a different probability of reward.  However, because we get the rewards immediately and the rewards do not vary over time, the problem doesn't fall into the second two categories.  That makes this a relatively simple case.

## Policy Network

Our strategy will be to use a policy network which is updated via gradient descent!  The loss function is slightly strange in this case.  It is defined as $$ Loss = -log(\pi)*A $$ where $A$ is advantage.  Essentially, $A$ corresponds to the reward for some action relative to a baseline.  We'll be assuming in this case that the baseline is 0, and therefore think of $A$ as the reward we recieved for the action (-1 or 1 in this case). $\pi$ is our policy, which is simply the weight that our network has evaluated for this action.  

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
# Our bandits.  The number corresponds to the z-score you have to beat to
# recieve a postive reward.  Therefore the last bandit is the easiest to win
# on because very nearly all sampled values will be greater than -5.
bandits = [0.5, 0, -1, -0.5]
num_bandits = len(bandits)
def pullBandit(bandit):
    # Get a random number
    result = np.random.randn(1)
    if result > bandit:
        # Return a positive reward
        return 1.
    else:
        # Return a negative reward
        return -1.

In [0]:
# Greedy Agent
tf.reset_default_graph()

# The Policy Network
weights = tf.Variable(tf.ones([num_bandits]))
## Greedy chosen action = always choose the best one
chosen_action = tf.argmax(weights, 0)

# The training procedure
## Placeholders for the chosen action and reward from the environment
reward_ = tf.placeholder(shape=[1], dtype=tf.float32)
action_ = tf.placeholder(shape=[1], dtype=tf.int32)
## Extract the associate weight so we can update it
responsible_weight = weights[action_[0]]
## Calculate loss per the loss function above
loss = -1*(tf.log(responsible_weight)*reward_)
# Optimize!
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [0]:
# The way we're defining our agents is a little sloppy, so we are testing it in this kind of ugly way. 
n_agents = 100
total_episodes = 1000 # Set total number of episodes to train agent on.
e = 0.1 # Set the chance of taking a random action.

print(np.array([train_agent(total_episodes, e=e) for agent in range(n_agents)]).mean())

0.79


In [0]:
# Probabilistic Agent
tf.reset_default_graph()

# The Policy Network
weights = tf.Variable(tf.ones([num_bandits]))
## The probability of an action chosen is proportional to its weight
chosen_action = tf.distributions.Categorical(probs=weights).sample(1)
# The training procedure
## Placeholders for the chosen action and reward from the environment
reward_ = tf.placeholder(shape=[1], dtype=tf.float32)
action_ = tf.placeholder(shape=[1], dtype=tf.int32)
## Extract the associate weight so we can update it
responsible_weight = weights[action_[0]]
## Calculate loss per the loss function above
loss = -1*(tf.log(responsible_weight)*reward_)
# Optimize!
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [0]:
# The way we're defining our agents is a little sloppy, so we are testing it in this kind of ugly way. 
n_agents = 100
total_episodes = 1000 # Set total number of episodes to train agent on.
e = 0.1 # Set the chance of taking a random action.

print(np.array([train_agent(total_episodes, e=e) for agent in range(n_agents)]).mean())

1.0


In [0]:

def train_agent(total_episodes, e = 0.1, verbose=False):

    total_reward = np.zeros(num_bandits) # Set scoreboard for bandits to 0.

    # Launch the tensorflow graph
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for i in range(total_episodes):

            #Choose either a random action or one from our network.
            if np.random.rand(1) < e:
                action = np.random.randint(num_bandits)
            else:
                action = int(sess.run(chosen_action))

            reward = pullBandit(bandits[action]) #Get our reward from picking one of the bandits.

            #Update the network.
            _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_:[reward],action_:[action]})

            #Update our running tally of scores.
            total_reward[action] += reward
            if (i % 50 == 0) & verbose:
                print( "Running reward for the " + str(num_bandits) + " bandits: " + str(total_reward))
    if verbose: print( "The agent thinks bandit " + str(np.argmax(ww)+1) + " is the most promising....")
    if np.argmax(ww) == np.argmax(-np.array(bandits)):
        if verbose: print("...and it was right!")
        return 1
    else:
        if verbose: print("...and it was wrong!")
        return 0 

In [0]:
n_agents = 10
total_episodes = 1000 # Set total number of episodes to train agent on.
e = 0.1 # Set the chance of taking a random action.

print(np.array([train_agent(total_episodes, e=e) for agent in range(n_agents)]).mean())

1.0


In [0]:
with tf.Session() as sess:
    print(sess.run(tf.nn.softmax([-1., 0., 0., 0.])))

[0.10923178 0.29692274 0.29692274 0.29692274]


## Our agent is... okay

When a bandit was much better than the others, such as when one had a threshold of -5, our agent very easily solved the problem within the fixed number of episodes.  However, when we altered the payoffs so that they were closer together, our agent didn't not fair so well.  In one observed example, it got fixed on exploiting the bandit that was closest to correct and rarely even explored the better option. 

We won't actually address this issue, as doing so requires slightly more sophisticated logic.  For example, instead of the greedy approach we probably want one that exploits likely beneficial path even if they aren't the best known path.  A simple way to do this would be to make the probability of choosing some action proportational to the cumulative reward recorded for that action. 

**Update:**  After adding a second agent which did exactly that, we saw a significant improvement.  Whereas the greedy agent was only correct 79% of the time, our probabilistic agent actually was correct 100% of the time in 100 tries. 

# Contextual Bandits

In this variation of Multi-arm Bandit, we generalize the first problem by introducing state.  Now, instead of just optimizing our decision for a single row of slot machines (or however you like to visualize the problem), we are optimizing our decision making for each of a set of n rows of slot machines.

This becomes more concrete with code.

In [0]:
class contextual_bandit():
    def __init__(self):
        # The state tracks which row we are in
        self.state = 0
        # List our bandits as a 2 dimensional array where we are trying to 
        # optimize decision making for each row of bandits.
        self.bandits = np.array([[0.2,0,-0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        
        self.num_rows = self.bandits.shape[0]
        self.num_bandits = self.bandits.shape[1]
        
    def getRow(self):
        # Randomly determine which row we are in each episode
        self.state = np.random.randint(0, self.num_rows)
        return self.state
    
    def pullArm(self, action):
        # Get a random number
        bandit = self.bandits[self.state, action]
        result = np.random.randn(1)
        if result > bandit:
            #return a positive reward
            return 1
        else:
            return -1

In [0]:
class agent():
    def __init__(self, lr, s_size, a_size, greedy=True):
        # Defines the Feedforward Network
        
        with tf.variable_scope('network'):
            # Take the current state as an integer for an input
            self.state_in = tf.placeholder(shape=[1], dtype=tf.int32)
            # One hot encode for our network
            state_in_OH = tf.one_hot(self.state_in, s_size)
            # Pass the state into a Dense layer
            output =  tf.contrib.layers.fully_connected(inputs=state_in_OH, 
                                     num_outputs=a_size, biases_initializer=None,
                                     activation_fn=tf.nn.sigmoid, 
                                     weights_initializer=tf.ones_initializer())
            self.output = tf.reshape(output, [-1])
            if greedy:
                self.chosen_action = tf.argmax(self.output, 0)
            else:
                self.chosen_action = tf.distributions.Categorical(probs=self.output).sample(1)[0]
            
        with tf.variable_scope('training'):
            # Feed in reward and chosen action and then compute the loss
            # This is identical to our prior agent. 
            self.reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
            self.action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
            self.responsible_weight = self.output[self.action_holder[0]]
            self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
            self.update = optimizer.minimize(self.loss)

In [0]:
def test_agent(episodes=1000, greedy=True, verbose=False):

    tf.reset_default_graph()

    cBandit = contextual_bandit()
    sAgent = agent(lr=0.001, s_size=cBandit.num_rows, a_size=cBandit.num_bandits, greedy=False)
    weights = tf.trainable_variables()[0]

    total_episodes = 10000

    total_reward = np.zeros([cBandit.num_rows, cBandit.num_bandits])
    e = 0.1

    with tf.Session() as sess:

        sess.run(tf.initialize_all_variables())

        for i in range(total_episodes):
            s = cBandit.getRow() # Get state from the environment

            # Sometimes do something random
            if np.random.rand(1) < e:
                action = np.random.randint(cBandit.num_bandits)
            else:
                action = sess.run(sAgent.chosen_action, feed_dict={sAgent.state_in:[s]})

            # Get Reward
            reward = cBandit.pullArm(action)

            # Update the network
            feed={sAgent.reward_holder:[reward],
                       sAgent.action_holder:[action],
                       sAgent.state_in:[s]}
            _, ww = sess.run([sAgent.update, weights], feed_dict=feed)

            # Update the running tally of scores
            total_reward[s, action] += reward
            if i % 500 == 0:
                if verbose: print("Mean reward for each of the " + str(cBandit.num_bandits) + " bandits: " + str(np.mean(total_reward,axis=1)))
    victories = 0
    for a in range(cBandit.num_rows):
        if verbose: print("The agent thinks action " + str(np.argmax(ww[a])+1) + " for bandit " + str(a+1) + " is the most promising....")
        if np.argmax(ww[a]) == np.argmin(cBandit.bandits[a]):
            if verbose: print("...and it was right!")
            victories += 1
        else:
            if verbose: print("...and it was wrong!")
    return victories/cBandit.num_rows

In [0]:
n_agents = 50
# Compare the accuracy of greedy vs non greedy agents. 
print(f"The Greedy Agent's score: {np.array([test_agent(greedy=True, verbose=False) for agent in range(n_agents)]).mean()}")
print(f"The Greedy Agent's score: {np.array([test_agent(greedy=False, verbose=False) for agent in range(n_agents)]).mean()}")

The Greedy Agent's score: 1.0
